In [372]:
import sys
import pandas as pd
import numpy as np
import random
import requests
import json
import matplotlib.pyplot as plt
import time
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [7]:
#### get all csv dataframes for trending
US_trending_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_trending_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_trending_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_trending_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_trending_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_trending_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df]
full_trending_df = pd.concat(list_of_all_trending_dfs)

In [37]:
f = open("apiKey", "r")
key = f.read()
## for each videoId, find a related video
def do_search_youtube_request(videoId):
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=50&order=relevance&relatedToVideoId={}&type=video&videoDefinition=any&key={}".format(videoId, key)
    r = requests.get(url)
    return r

## given a set of videoIds, find insights (statistics, tags, etc)
def find_video_insights(videoIds):
    print(videoIds)
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={}&key={}'.format(videoIds, key)
    r = requests.get(url)
    return r

## call this with 1 country at a time 
def process_youtube_requests(videoIds):
    df = pd.DataFrame(columns=['video_id', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 
                               'views', 'likes', 'dislikes', 'comment_count', 'description'])
    relatedVideoIds = []
    for videoId in videoIds:
        try: 
            response = do_search_youtube_request(videoId)
            time.sleep(2)
            if (response.status_code == 200):
                r1 = response.json()
                relatedVideoIdItems = r1['items']
                for id in relatedVideoIdItems:
                    relatedVideoId = id['id']['videoId']
                    relatedVideoIds.append(str(relatedVideoId))
            else:
                print(response.status_code)
        except:
            print ("Something went wrong here! 2")
    random.shuffle(relatedVideoIds)
    videoIdsForInsights = []
    for i in range(0, len(relatedVideoIds), 50):
        videoIdsForInsights.append(relatedVideoIds[i:i + n])
    for videoIdList in videoIdsForInsights:
        videoIdsStr = '%2C'.join([str(elem) for elem in videoIdList])
        r2 = find_video_insights(videoIdsStr)
        time.sleep(2)
        if (r2.status_code == 200):
            r = r2.json()
            i = 0
            while (i < len(videoIdList)):
                try:
                    id = videoIdList[i]
                    title = (r['items'][i]['snippet']['title'])
                    channel_title = (r['items'][i]['snippet']['channelTitle'])
                    category_id = (r['items'][i]['snippet']['categoryId'])
                    publish_time = (r['items'][i]['snippet']['publishedAt'])
                    tags = '|'.join((r['items'][i]['snippet']['tags']))
                    views = (r['items'][i]['statistics']['viewCount'])
                    likes = (r['items'][i]['statistics']['likeCount'])
                    dislikes = (r['items'][i]['statistics']['dislikeCount'])
                    comment_count = (r['items'][i]['statistics']['commentCount'])
                    description = (r['items'][i]['snippet']['description'])
                    data = {'video_id': id, 'title': title, 'channel_title': channel_title, 'category_id' : category_id,
                           'publish_time' : publish_time, 'tags' : tags, 'views' : views, 'likes' : likes, 'dislikes' : dislikes,
                           'comment_count' : comment_count, 'description' : description}
                    df = df.append(data, ignore_index = True)
                except:
                    print("Something went wrong! 3")
                i = i + 1
        else:
            print("Something went wrong! 4")
            print (r2.status_code)
            print(r2.text)
    return df
        

In [11]:
n = 50
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()

In [ ]:
## do following requests separately with a new API Key

In [12]:
not_trending_us_df = process_youtube_requests(US_trending_videoIds)

n-Ej2EtE744%2CBD6HUuvHiM8%2CJoYNhX15w4k%2Cd3WiRunOsWY%2C1G8YP-EbfJc%2Cut26dVlDSuU%2Ch_7d-m1ehoY%2CVBnLmP0Ia9E%2CXqOu2T0PJMs%2CygCxoUqdrbM%2CUNwKNut0id0%2CVLAu7eeaGYI%2CsaOItOSKZBY%2CUtwwCpadfFs%2CrLBNowABzA0%2CduXW_wPTw0o%2CvsA_L1t4vXY%2Cmv6jrpbbyJU%2CdqAIxnFsBeg%2C11H18Xq11NQ%2Cg0RYVvcqYUM%2C0U_UsNMTTS4%2Cmc2w-xnfmc8%2CxtS6JxTpOF8%2C2oGE375BUgI%2CjhW19RFoWgk%2ClKacqMdMfi4%2CdWttEUMFXH4%2CT0jM37coZPo%2CrO-ttUAiVDw%2Ch9fqgarR974%2CDo3h79BIkSA%2Ce6RVi48aOho%2CKdLbZdH0znM%2Co2572Ar0VNU%2CbMrRN99DYKU%2CbOId9TRRHH4%2C6okzxazlFSw%2C0lF7hpVCilg%2CFhskvloj1gE%2CKX1bQ1sGiIA%2C6cmMYouVsXg%2CWN1WSTGqfj0%2CHXIJcIivMHg%2CFo5y3Ydyhtc%2C9-XnufbXpak%2CrU7lWj4arVI%2COlmAaOdK6nk%2CYlpXoQEqBWI%2Cc-nEjZJVjvM
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
NtNUAoNSbuY%2CJTCckccilds%2CwylMwU1wEOM%2C6BRyjhmOAXg%2CPEpgKvq5DFA%2CHYk1nnm6mRc%2CBN7rNfOsBJo%2CSTVcwLIQF-I%2CuZeGPAlNkIE%2CXJv9Lujp0Qc%2C5qzA7bzVsO8%2Cqhb5kk8fvbs%2C7F4r_VBP2m8%2CFqnWEWYp9DU%2C1yv_MXNYbAo%2C6xURhmqwhpc%2CKyT1yA

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
P47pPrBDras%2C85nBQ_AoRwk%2CKUcIBLBKG84%2CgwtZ5XQMb00%2CPNHp4xGI09I%2Cu_0V3PuZV-U%2CCO6tM3ry2TQ%2CBn50X23fZAg%2CtbodAlifFsA%2CoyC4lLTOyL8%2CQ2hqcPnIWiQ%2CIqTnBmYXeqQ%2C8s_7IZDd6uY%2Cvm0kyrI-B-I%2C1OZrb4A82F4%2C-zjJpFYtx9s%2CRQUuqbzQVsY%2CqHO9gARac-w%2CxsqInns6LXQ%2CRV0AgxyPozs%2CpyCENfiLhP8%2CHubBUXjHoro%2C4xqEd3ElWJA%2ChyJwdGVnb9k%2CQXzoJRVk5wI%2CpgL8jqcABSc%2CHtylEh0Y-6k%2CRrfsRT8qk4U%2CdCsr0CNqB3g%2CQXuTz-5N1Aw%2CpLBcp3nJlFQ%2COfS5MI_3_WE%2CieAtWSh2i_Q%2Cb_i2DDUwlHU%2Cu4o97odnH2g%2CQVz0IlrzPEY%2C_AWeetj7pf8%2CDORFhTdRhFQ%2CjQWZDcswcMI%2CC0gFau8o9Aw%2CddLMFf3L4Q4%2C1gqJWglis6s%2C37lrPDfZiIs%2CCIwrCIe9wsY%2CamsUNARAwQE%2CSlskTqc9CEc%2CvXoqZ2oyYKk%2Cls31TkGSQGA%2CJuj2EMofpII%2CfwrgFoTQbP0
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
cthb3_PW-Zg%2CkUMYT6tozEg%2CS8XRgnEhMZ8%2C_M0m1whfe3s%2C5IsSpAOD6K8%2CgqbU0Ig-iiI%2C3lEfl2-k_tQ%2CRYy2Bu4-lcA%2CzNNV1nlqQAU%2Cyd1CUGVO

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
nSKZ7OJxgWw%2CJZ2fRE_nkak%2CAv-FkiIqmS8%2CQLXrq6BZcxE%2CcnNbCAqPBRg%2CefxiDBygvdg%2C0Fi4-cWcBPs%2CB5SKOyyNzEg%2C6it5HQFA83s%2C1zdZG6KFM0g%2CVlOq_4lmpzE%2Cr1ODk_tcQks%2CXrJl34zJhDE%2C9WCcBwnYaVA%2C-3uKxjULweU%2CqEO2z7ds0aE%2CoQsZdy7N0Lo%2CS-BA4PRUVMU%2CrdeQT7KkqM8%2C1fUO3Xjlohc%2C16v79ilqc-E%2CF2RyyCdJZCo%2CAopb6YZyeFU%2CtdfLqRNmnSY%2CyIq-N7m0upU%2CqSLHVgaI3rc%2CG5kp7dTYHOo%2CvD-tWNJ5Orc%2CliGwcuFyaNY%2CGNPyVVLNVyc%2CKpHT6lMz2Hw%2CcYIiyr59b-0%2CCVo225pUaSA%2CXM0udsRZ4FE%2CzNqSvg5qwGw%2CY39upK06kEw%2CVPyuUtb903U%2CTYcEf6liwsc%2Cg4JG0jCa0ko%2CPX5-XyBNi00%2CC2NNujJKJL4%2CVEl1i75rlRg%2C7VwY_ZxG1AI%2CiYAIPs_6GLU%2CFMABrdI_L8g%2CRecdrgB-Odc%2CxQ7-ingO9eY%2CdjHRF7Wxmxs%2CUWw9Pnv1-8U%2C5zugfaPFJ9s
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
xPR-Ej-ENbk%2CFAOC-0Le8EE%2CNj0RnbGBIkI%2C_O2_nTt1N6w%2CiXGEbDVgcFk%2CMgLO_ojVcJ8%2CHj59uPPs-uk%2CDY9VE3i-KcM%2CVEVpeTHOMvA%2CUpm9LnuCBUM%2Cv-4g0c8NCoc%2Cxpzi

Something went wrong! 3
Something went wrong! 3
NX1xzmKK62k%2CKwGxEnYON7U%2CBdGeXcJ-hfo%2Cj5Zigh2Txsg%2ChI5jggQTAsM%2CjwkNAJBwoEA%2Cjl17CYYSzUw%2CNZKXkD6EgBk%2CAhYWw_brsS8%2Cif6Nvn2ZzSI%2COC451cdFLoQ%2CiafOP1aV3r8%2CqM64D9tA-cU%2CdW6NhWv0hgU%2COCzuiORI2lI%2CIIeGWpHwKw4%2CPaxYlED9h2s%2CUx21C-1k7l8%2CnJU0_XfEoXk%2C1nFcXcPldzY%2CW1hQHZedRSE%2CG1VEccH7jJ8%2CMdkvi8hwGYQ%2CKu10v_Vpwoc%2CiDR82qG5uzs%2Cwm32uDr4Hcs%2C-VDSYYArZPg%2C00HnxWOqBWI%2CmQBQ0f8KdqQ%2CDTRb6G9c9wU%2CmNCfR7oCWJo%2C2riLrz7DCpQ%2CSsJTWfWYvTM%2C78HQRu3naaI%2C8M6gvYlVDz4%2Chz63EfGgJjM%2C-eVd2Ugk9BU%2CTXWHHNktmh8%2Cok99C_cLlOY%2CO_vFWVXUvo8%2CeQEGPATQe5s%2CQPZ0pIK_wsc%2CfzZnawHitMQ%2COXxrinLM8Cc%2C1yPnl9KFEsQ%2Cv3uGJGzUYCI%2CruYx3gevWoE%2CkEnLbEtr9kE%2CjM9gj-JWJVk%2Cy6Wz3i_BYUc
Something went wrong! 3
Something went wrong! 3
lMLo0E66O8A%2CaS11FR_tKLo%2C3WcQQgrG614%2CtPTXbAAh_lU%2CAfT4LPilNG4%2CQkNHGGvvZcQ%2CXpw0ts8RTfk%2CVxE7Fi6jH54%2CulFMFi0pHDU%2CgEi5RfGDmVc%2COZb8Igc00qQ%2CKXaSlHoQMt8%2Cr4DD6ZrZ3_4%2CzNjO6arjuAc%2Cf89H24nf0M

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
JmWfY4nb6oM%2CjnYX5ZL9y3U%2CvNj0psXnS6w%2CTiwxeUR2ZSw%2C2DKu4_MCD-M%2CopHx3_SdKsE%2CGKHgOY18OP8%2CfGCMcu2iei0%2Cdnjb38EQM84%2CTeRDSwEXw9g%2CkZEO3QPWFx4%2CFRI4TZDQjck%2Cqjbr4mbsl54%2ChCmy01pvO3M%2CCHMpzGbL_Xw%2CUwGNDIlhWj8%2C8cIzehJfNyU%2CG1kItProe5o%2CEsBfpHNtCtY%2CkD-_iyS95AQ%2CR9oqi6HteJg%2Cj80eR4-L54w%2C8IMppGt8NsU%2CvXfXrRp0xrk%2C9ABvptRayb0%2CZbFdlldYiYI%2CjFdRIjP3MIw%2CKyNukw1rltE%2CDzl9seZlryg%2C-eHxLqT21r8%2C2H5R0bdblEE%2CkiFDqOYfgYk%2CKbGV1KEXds4%2CKK3FQXObWEs%2CMdEV-jWMGWw%2C45LTdsV1cyQ%2CkY9a57UtWgY%2CA0WyGWDcC6E%2C8dqEGxpwZ1g%2CL4LBgLH281k%2C8KascilkK7M%2CeACohWVwTOc%2CibA6rEGO7iY%2C5EBdps7VBxQ%2CQHRnvB5rruc%2CE_IxaI3jF1Q%2CvtFI4l8IVxA%2CP13CPNNLEQg%2CoFrh8LozkAA%2CeX0_7W1K_Ng
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
ZJ3QQ-SNCso%2CnpRcdl3f6iU


In [22]:
not_trending_ca_df = process_youtube_requests(CA_trending_videoIds)

S-sZMprG58g%2CShZ978fBl6Y%2CoKAmujgS4mo%2CptnyAECPXLQ%2Cb-lprjWfFN4%2C6-FaY_kd4xg%2CKdDz7HKc4F8%2C5j6k2MPYtRY%2CKP9JelGZg38%2C9mYVi7WHyiU%2CS7A9escW70g%2CqyL7TG6SzDU%2CVpozPu3TXfE%2CN0MgHSMDALU%2CWYzPTDZR_fI%2CEwMnPgbhEM4%2CyLbAPK0LCfk%2CDYduGBjjQ68%2Cd2aCeJo-LFg%2CRu20fHqWAD4%2CMun0ROv8MiE%2CNnBHG1SmI-Q%2CAgFaljoriYA%2CYMzKs64FT3s%2CnORg_aXMsmA%2CXI7rZ_INi7c%2CSWfJm5sLVCw%2CSPhUulCz1aE%2CM1Yg8kzcQXc%2CE-VAVEGCwGw%2Cjm7E4vzvdJg%2C7GtlncnFRWM%2CaHmmA3SCXVo%2CllRjRPgG0r4%2CM7QSkNpu3XA%2CZ4dAMMkqPoY%2CMkWGimTCnBU%2CSYccDd2zpJo%2CIh3pRKgGv5U%2CxoAyXkahcXk%2Czn6t267flQs%2Cbi2HmIdL2Dg%2CAdHt_3wJASU%2CosmKFXD_WlQ%2Caasr7bqi8Vk%2CqzfMqqzmQmk%2CzDo0H8Fm7d0%2C243wWj9haKQ%2CbCEGK_1yLJo%2Co8W-UIco3yk
Something went wrong! 3
Something went wrong! 3
0J4KGpUDRW0%2CP-DhwN87JDY%2Cr-GAu52U7rw%2CusRWa3UtXNk%2Cqk2WMmiiVFE%2CbasFyoMSjds%2CJE6bWTtmNpU%2CnI1eT9jcsZA%2CfAhhcl-bjPc%2Cv3fvyCzRqfg%2CAn_DFdNow9o%2C5OdN9OSET8A%2Cr5P-f5arPXE%2CeRsd6M9vByA%2CTNRldP18bH8%2CWkw8IAMkmbs%2CRqGiXnkKB4k%2C2CICCBjnylM%2CO6

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
0R_vu73qqIg%2Cezi4yBxcmNc%2CyOvoivYuo-0%2CXv1r1wWKcMc%2CVaryO7wOic8%2CTb4vf4lFpuM%2CqjTu_8hklUw%2CBfFC2gGGAGw%2C9gIOOxlV0Ys%2CiUnobJp3eH0%2C8V6f3fJE3VE%2C8Cgy39KP3N0%2Cex_sBZXU6iU%2C-ncIVUXZla8%2CUFFt1F5qumY%2C717JnxZVQkc%2CZgy5fX-VPCs%2CHw6NhAsng-s%2CmH_2iYhZmcE%2CSxZaHnKSrFw%2CnGSsjO_USts%2C4QD2CDGf8fs%2CDUWi4jWydnY%2CZUpWMtsWC3E%2CipuU0hHYiF4%2CTvpAdCxE1Jk%2C119msSY-Nuo%2CDsVUFXVx2pQ%2CtNUsoangGFs%2Chu2hjtLeUG8%2Cr5ZWnS-nytI%2CKmEakhzw3k8%2CCqTyOCSXO9Q%2CdeaozBuoTcw%2CaFlFvDvhlVc%2CvJP4Pe5kH0c%2Cz6KkR4h_H6E%2CQE6PvNohffc%2Ch--P8HzYZ74%2CYravW4qZHig%2CKocwh6WySvs%2C5bm8QRjpPBs%2CuEfMr_QSzCQ%2CaJOTlE1K90k%2CeUEUzP4mxjo%2Cmhi1UpJmvrs%2CHRjzvNvrhu8%2CrpZG8wh8d-0%2C_obQCC6bbho%2C0URIsIPnsrQ
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
p3VF6acYG7I%2CEsBvX4-4khQ%2CHtu3va7yDMg%2CIhnI1geDce0%2ClNbKhl8Zkmo%2CY5cMnT5hW-0%2CkG4dThGHFpY%2CnoPRsTH9Tf4%

Something went wrong! 3
Something went wrong! 3
xTrJJG_3ow8%2CpIOFDH9OblQ%2CvH7ASHJO6kQ%2Ci_jgF-S746o%2Cp3VF6acYG7I%2Cpa14VNsdSYM%2C0-rK1emidMM%2CfKacULQUyDQ%2CB_YsJVpxA8Q%2CDYPyJ0EXXLs%2C4immiFky4hc%2CfRG37dYhTNw%2CG5R7CMggsx4%2Cx1cetsAA4Qs%2CzOnoE4AYqHQ%2Ck4GAUb9WUW0%2C8faNQGNHq6c%2CmnT4MDdEazs%2CDxNGCLA9W2Q%2C3FoSAHk7DMA%2CBbO9K0tdhuU%2Cdto4koj5DTA%2CZVCwzKzPTLA%2Cd7cuNRNwSDs%2CS17ShHKgQ7c%2CDcqMp_D5pdE%2CQE-5H5Fzo8E%2CUJeTKIg-qNM%2Cof1KYsVs3u4%2CQ3HFVuZ0Quo%2Cg0RbhqalZZk%2C28_zmf0YPs8%2CCl5r40hzij4%2C-XHtUScg-gA%2C0QxbLpqazI0%2CHcNcOnIkQQU%2CLS5KO37Sd3o%2CDlPQcw7srHM%2C355MEuxjcNw%2CxRXcZVVPJKo%2CJ7UIb5l_TFc%2C5Y-DcLF28y0%2CUrUJyKsLQeU%2CIbBU0uiHmsc%2CN2eoRU0mmXg%2ChRmVJHB0beo%2C5-AV1FDAU5Y%2CIl72uEjs4o0%2CPjTU0DmBWiU%2CpAIun6ik0c0
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
FX1HaV_qjOM%2C0cO0RNCoEWM%2CxCZWnRvQoIU%2CFszemOnRVOc%2CFA64SGvuRWw%2CRTisrL17org%2Ch21nhcfKEkc%2C4uBEjuGCk5E%2CrEL3W9xNn9U%2Cg9iL9QQ9zy4%2COyAT7tffnt4%2CSKxQPaRdI4E%2COx8CKkM8_6k%2C

Something went wrong! 3
Something went wrong! 3
MThBUZ7nU9w%2C3RDSYz3eOYg%2Crl2Vazf84f0%2CG-pr6jtSteM%2CJRfuAukYTKg%2Czx5nzTbh2pk%2CShRVs4uPZTA%2CZkAVGUNEfkE%2CiM_KMYulI_s%2CUvIkojfQDxA%2Cx1Y38eWTu98%2Cn-ZFHHZKZh8%2CJmf8AwOac18%2CMP_nWuLYpJw%2CI67AwNlYfv0%2Cfi_MA8XB1yI%2Cn1y67Nywyyw%2CGaNANGH4PIA%2CIrICG1c1tWY%2Cp6M0QeZhcog%2C96YOZOU7ggo%2CbPfzNYN5lvA%2CJE7tC9F5oVs%2Cue63lwOU-h4%2C9Hd4SGyF3TE%2CEsX_W1uzSG8%2CBRXdjc5yxVs%2Cw1mopOZFcr0%2C4QDDxmtbUsw%2CynTkPCW2P_I%2CbLHcxEtVMao%2C5S05A8Y6cpA%2CxOb98jleRyA%2Cpw2A03Z91FI%2CwtoG5N3v3Ak%2C0wysCoAm2Yo%2CJEeCxgv9k5M%2C3wkgi7vxrkM%2C4jVUaT8UHtw%2CfEbed_DvQZI%2C4U_Oh6pziTo%2CA1poQzpOf94%2C21h_DDTa4dg%2CD7wmMZmMinM%2CglVpIOTvtd4%2CwxlvICjOLIA%2CoyOPphro1eg%2CbasFyoMSjds%2Cop45eGMlptY%2CigaNXJWzG9s
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
kNCjUEKPsEo%2CIxxstCcJlsc%2Cz6ZNrr_Qs7o%2CSqoZlsZYrd8%2CVX9v93OmHb8%2CFyRUP9z1lX8%2C2BHUtwMj7vA%2CzFAYNWddDt8%

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
xfpT6bxzU5c%2CLoKHGTFv-XM%2CgYCVMxQ-jWU%2CQYksM59LgpU%2CQ5yNeLqiJ9U%2CbSiKMZs419Y%2CS-BA4PRUVMU%2C1mzi6pm8PBQ%2CNpjoPhDlkJE%2CCwO_JI6Ar48%2C5QIq7xgY7ZE%2CDVZgFSlFRp4%2CH4xH8sw0Eh8%2CUaygIvkQnGI%2ClIsfer_52L4%2C1cliI7tQilU%2CsgyywtrehiI%2C5kNy7-IYTv8%2CqDzY7_EKQoM%2CU6bpbdnAWjI%2C9VKIm0sWW4U%2C34CrjMt_LCM%2Cjhonpfb98Kk%2Csqom2VSho-E%2CHDOWBEhC5js%2CulFK-xhtIzw%2C_lu_Hgw60Ns%2CQeviyBHqR7k%2CD9F4Fnle9NY%2CRKBvVctcQE8%2C32g1ByDRzO0%2CqalaGsVqWjE%2CvlnKgxFochU%2Cy8c2x9Rk5PY%2C044VmdKhKzc%2CF1ZrIyEAFqk%2CcnNyUGhXTsw%2CwWn0YFBo-og%2CysYYxkUVLJc%2CzXnd0_KKzJ0%2CX_BzUWLgKbA%2C-SB5oUcx_iQ%2CnslDUZQPTZA%2CmmkCS5eA4f8%2CCAaKfOsju7s%2CziIiTRw8PNg%2CsHdmQPkvLew%2CITQE0vEH63o%2CzyEcfMTr_w8%2Cvy__i3i099g
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
614QePwQrMc%2C

In [27]:
not_trending_de_df = process_youtube_requests(DE_trending_videoIds)

KSdnIfgz1Jc%2CDJ867BdFUCE%2C0h2cJkK4B30%2C9oixChs-kT4%2COH04CKBqGGA%2CvcsYqR9_Smk%2CLoCVhd7Dr_o%2C32VPt29srVs%2Cdrh2HvE4oYw%2CQyVdIZ1Vj0k%2CrrWZSgshNz8%2C-R_CQ2vfvuI%2Cb0LeoEPfvkc%2CVQcwEc14tq4%2CLghXfuvMkww%2C6I7n5UfemgA%2C6-wtdpoLHUc%2CCFeMgXqrkTo%2CpXZgx8G7XKA%2CbVKgGQ4_af0%2CCooB70w3qEo%2Ckz11mjhVbGo%2CT6-CQXfkmkg%2Cz_HObp4xZAI%2C8zplEZjM5Zw%2CGy0F3m368EY%2CrywXOXQXgug%2CINAAyBPtBIE%2CbzTo8KDw2xY%2ClcrockX19M4%2C1YJsnpAHFb4%2Cc5-3ZnohNbA%2CHdjU-wcqNLM%2CFroyhaobQuc%2CRhGtEgxTLz8%2CNb_BNHE4TGY%2CB2IcBO6TEbA%2CuDzl5PG51uI%2CqsYVy9PBQOQ%2CC4YR6AEnGpo%2CCdBQ1mh8kFo%2C3dckH70S_o4%2CrJiCu8DwhHI%2CUYexfQ6edGo%2CNCvGRVh9Wx0%2C8V-I_nSoKnE%2CqSghVWkTGXI%2C2YGxrZwNe-U%2CNQHHfdRwja8%2CkX-EHNlU2FM
Something went wrong! 3
N9APktNiKgI%2CWsrck7F89Ks%2CiQMdRERSlqg%2CpuZKfAI5Gfc%2Cg2vSXq_b1Wo%2ChB2I43FGgM0%2CZt5lSRRzV2Y%2CqR59RRW-kFQ%2Cl5hWytnGdQU%2ClS7_p5vUIBc%2CXHRumZiLR8w%2C-U9DzRb0VM8%2CfL1JcYYBWrI%2CPRWwpMdHwK8%2CR33aBb1LNhA%2CmVFccTEm0kk%2C8EMaYTLF2go%2CF5zSw1ExmwA%2CgpIOhalO3fw%2C18HB2N4l2ZU%

Something went wrong! 3
Fj79-qoMJFE%2COYIydcqrGeY%2CODvV3M5l8d8%2CroUGbBnW93k%2CVKdpyQAepjA%2CmE2y2vr22oI%2CbvJ82K5o3u8%2Cqcxq8sVJyqE%2CPVHjpXJry7A%2C7vKhg9q01qg%2C4MpeJA0umb4%2Cs4IWgUjA6oY%2Cl5_foDaatxM%2CRXGooMD4lXc%2CeHmyu-4Kc7c%2CeEDFwwddLJ8%2C5p4EuB94N8U%2ChYtl4JZDXsU%2CuJ6Y4jKaC4U%2CHXv3_EG8xU8%2C3fbyynANl8g%2CGA7bH3KGcsA%2CMmtbHRsySZg%2CxHPC8OgfJ38%2CmjAviBlWvmg%2CeHPDqPb41sA%2ClUrxqGl1fxs%2C-c_6n62uiEM%2CXQat6rNNbdQ%2CIeA7lvC1ego%2CqcxJHo46Xfg%2CHG5BU_1RMSw%2CW2lLGRodDww%2C7jJ9VC8toDY%2CZFikZXcJxS0%2CGAHkH2fB82I%2Cj-Snmzg0LX0%2CChP3t0sZOYM%2C657Fgs5RGT4%2CKKxfRyoVZtU%2CDCWdTT1GZKY%2CRzLbZ05sWp8%2CEBgXzg2I-0s%2Cls31TkGSQGA%2C0n426mfyTkU%2CcBaskZRxB0c%2CfcB8rLfN6NI%2C-rm4vXsoLXU%2CahhJYKYCsUk%2CC8CleONuaPE
Something went wrong! 3
Something went wrong! 3
VvpfDikL1Ro%2ClqjBwTSbW2A%2CXpqtJVwMe60%2CWO3SVo9FkpM%2C1gnPFCy7D8E%2CU31xEElxq8M%2CrrWZSgshNz8%2Cs2KJr0rq2sA%2Cd8M45jmmDfw%2CaPFNL2bjLBU%2CiHQ4-bhKQwk%2C2ObklbyjtBw%2CcryslZqk3us%2C6269dp6Menk%2Cfef09deAveQ%2CwIqv_DR6ij0%2C1FC438

Something went wrong! 3
Something went wrong! 3
B9FM9uZQHzQ%2C-tS7Yecuqwo%2Ceh7u32ImWn8%2C_7V0eSgt9TY%2CNSiaP-MdyYE%2Cs4kxFNnjaoo%2C8_3Ds5WuSw8%2CYzsf9SECcEo%2CcBiGhkFKCNQ%2COYLAssa6ZYI%2CSW47KkAiTJY%2CVjLhP6CskYE%2CdczQntLA1L4%2Ckl6klkLNWVw%2CCzT_KxWpAaY%2Cunk75876GZU%2CN9APktNiKgI%2Ctc-YiCZMakY%2C6kBAgJj_v-8%2C2954-SBCixQ%2CCqMrfeX6qnk%2CSz4guSQhCnA%2CFTLu6oNekOc%2C4mOUgs7ffWE%2CgW6jufY9pYs%2Cu5taZAxigug%2CtSSYqWHV-js%2C8tvJz4tSWao%2CO16IB2O0thE%2Css70FiFg7j0%2CmewW_pRSgEc%2CpSk1NjjtuDM%2CQ5GrtR4c3VU%2C82fUR14-nIQ%2CiLYmP9Di-5c%2CxR3hrZS2c0A%2CKy9iKNJ3hHg%2CjChijVy0GTk%2C-MoyDyXThxY%2CWrctAiLScSI%2C3j_6cLm1b90%2CKjI3TUc4VvA%2Ctm6hg6dh5n8%2CPDJa8Fgc2pI%2CNH1XAMjC2d4%2CP1Dafy4L_t0%2CRfiqJptpGEw%2Cv0BpgYh9mm0%2CA9ds7sWlA64%2C8TGcy2jxrhU
Something went wrong! 3
Something went wrong! 3
8u2W8BN1l9M%2CHKmqr08lRR8%2CRhxHjujpRlc%2CL6fOkMNmqF8%2C-f02fJEMmgo%2CL0GP_m9QJ2E%2CKwRx4oK-N1s%2CxG3BLxA3vjU%2C57-Ecyj4H_k%2CBWBS6w1kLKg%2C9ToYyBxVZ00%2CQtVL76gh09U%2CaSyoW62Pz7Q%2C6RT10PoiPvs%2Crw2sLl8kXa

In [32]:
not_trending_fr_df = process_youtube_requests(FR_trending_videoIds)

nkavwjbUYiQ%2CaxFQKlW6m38%2C3uukFIvhAzg%2CxhPJSfASQp8%2CCqs9Y62l8ic%2CmtRHO9N5yIU%2CNQKCIeFmeoo%2CQLH54rfkzps%2Ct99VPumx6nk%2Cy5qHovcQH4A%2CVDezJPehj8w%2CI82qysNXwF0%2CDxi2FRaTW7U%2CmhqvGWrqcEU%2CVhxeitxAnu8%2CTrcTpqlA7oQ%2CXBmmy5tBiWU%2CvRyFKlJRrZw%2C0emGqqO59HE%2ClwpUA8Q0jsY%2COAvxBkQC9MY%2CJ62ovlR4DVU%2CPS86JVlQf7o%2COX8o_Aao-uk%2Cu4_6tUqKPCQ%2Cvx6B97Yik7c%2C1ihc0k2E9ro%2Ch8YtYDv-MDA%2C5k3eEWLO-OE%2C_VUd4YPaKWc%2Cq2yoauNrnVE%2CKA07E1y5OPg%2Ch7u-nxzZAtY%2C517dG0ddxMs%2CfiujqGKI8dQ%2C3kpAFGofUD4%2C9e0jryu4Ov4%2CQYE2yIijan4%2C6r0cQQMhB5w%2CfBuyhVbAdIg%2CmS41xRKvEMo%2ChSso-_bGtPc%2CdScslgkAHJU%2C0sZJB75molM%2C5x3YVM45vDA%2CnfumDgLavns%2C_sy06aK0K3I%2CnLUP-ShxQz0%2CddmNTsoRsVw%2CeSiByNrxVF0
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
egD3nAre1MM%2CYxIyCfczjP4%2C7Cr8PKyol7s%2CwLcWf8fN54o%2C1lJOpFI3HRI%2CZ3ya2YB69IY%2Cj1hgDPAVM68%2C69VKg_tZyKk%2Cs7IEm-Gf0Lo%2C7vwrYr-Yvq0%2CLqzFn5OUIyQ%2CvXe27v_OflE%2C5k5SyvEdBLs%2C

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
hXnExxYOCBc%2Cc9R2JHbzsL4%2CH1DPTpwtDRg%2CWKY4yZQj3y0%2CGeu6LQ2MqqM%2CxcDlIvdj2zs%2CWQAIuOKFydw%2C9jKD1JJbKaM%2CQ2JXf1xedjQ%2CvfK6m01OF8I%2CM2adk5mdicU%2CYpAZvBK2A2M%2CXAJHNB_bT_Q%2C31OMf1H5Rk0%2C6iWGA7OABXg%2CJAGGMdGrfmY%2CnwCWutYqlvo%2CBCpyfFc_OmE%2CogGD_-xtXcU%2CbCAyz9VCukI%2CIlq3DPR9FIE%2CTr4MfoVaRC0%2CpiZlhGnl5BU%2C9lcgPrefg3M%2C0TBJF7fQzs4%2CksGPamfKnys%2CCevxZvSJLk8%2Cb5atq_VZd2M%2CBnCbNfoj6S8%2CR8-vusknB4I%2CwXLM6hP3xOI%2CHwf-mwhupBU%2CCDvtDsRF3Bo%2CPEof1Me6qNo%2CI1Fax8S7GaI%2CjuBuFsgxipw%2CVnuyppWmPPo%2CnndnElzb5cE%2C4992UETrwk0%2CvnIUbJhj0TQ%2CRgq5iUwK5XE%2CI1w1k6UQ8gc%2CvHjTT5drBk8%2CZfWP0nMW47Q%2Cw_MUjdII9tQ%2CqVjM0owoGkQ%2CLQr43TA5_AY%2Ck1BggYoy444%2CeGeSuiPkT_o%2CaiYIPZdSXzE
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
EG5hPr7WwSU%2CTPezN7VNoAI%2C3Z4a3tZBxhk%2CJX3uxfHu4C4%2CHpmZuLriCws%2CTBKAA8BHFP4%2CP9ztY31SJV0%2CopVlj_ViGKQ%2CSf_K1Q7eKKk%2C1wSjLjJh

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
TaENsJlPMQk%2Crew3qfQV6jU%2COR09FqrEVgc%2Cu7Na6HiT3pg%2CvWq3jvMTFcw%2ClRbcQ7Th1Xk%2CvCOiu0Cw6gE%2C1slasHz7jAw%2Ccp0a1TjChuc%2Cd6UODLcuFZI%2C2MhIx8lsrm4%2Cja4aTfSco0s%2C8AdoWNI2XD0%2CN8PBa0_xC3Q%2CazZ4_BXgacI%2COXVMzlVCnMo%2CRmAidNzk4pQ%2CYIeEuPKom1Y%2C8-hdlogS-vY%2CR382BM08Ia8%2C4tloXqEJC6g%2CFyl8Qqm8mpI%2CE4oWKpgThZ4%2Chx4xMt2oM58%2CBS6eUUI9Ecc%2C8ttnU9roWZM%2CUmJ2rqVvmdU%2CfmydVwwFtKA%2CRKtuCoqjeDA%2CyRoOCv26-CU%2CE90h5HlaAM0%2CbYum-4BSDVk%2CPEwkjp8bTTE%2CJGlxBfd89Lk%2CsEQn_J2qNug%2C4FZaBz5eSeM%2CApTLDhYtMIk%2CP9kfYrh-dEU%2CE_mf8vpHY-w%2CfRKXdjSVW1I%2COO_i1zc3L4g%2CFb2M4MEW9gM%2CWkd_-CIVckk%2CvTPv2_9o9Vw%2CpFtdfqcFqPk%2CVq8YnfXR1OQ%2C37moCZHZYbg%2CkTZV1lmF73o%2CGysdeRlBIoQ%2C3N3c3ZX6hoM
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
0dajI_AqiEg%2CvleeDaaXhZ0%2Cu8VLBisyRfY%2C3fB2urJLuZw%2CZ6VAOGXbvZ8%2CzWcubbFeads%2CkIJjFYmo7e8%2CCEWhpfKwH5A%2Cek4Nqb23-LA%2CvDUKELHm1Sk%2CIBi3d7aFL54%2ClCaUeBQatGc%2Cd09ZePCk1IA%2CFLViZqgpReM%2CQIKjJ1PieU0%2CITJ4RDKxyOA%2CXxrL7scaSac%2Cjvwj841ab98%2Cm4KJHGtmcI0%2CjpyOcPU-o8k%2CeIvgWc3XyCg%2CgDGX8xkmcv8%2CAtw0UnP7Ee4%2CSkokU-2QuBc%2CRZtQwyfjHiw%2CgYC_Zh9P5MU%2COt9QYwThzsc%2CK0EV7wlMhx8%2C7_Q3aNWQhnE%2CHMxvpPT4jZ4%2Clz12RDmPaQ4%2C8WjskVCqZzw%2CpZZFE0c2dbk%2CKiaMLxSfbzk%2C2GjER0UVwcg%2Cw-ftGqpjZYw%2CJGPDYSgr5eg%2C0hWNzpEkcOE%2CtNe_nPT5Py4%2C5zgElEm02k8%2CcueFFub-OcE%2CFPE1_7dpqgM%2CTd9_GiiDWE8%2Cq3fGurXZVF8%2CQmw_MascLbs%2C9uHa9yaY_cA%2CJyBvH0gaNhk%2CPjug6s-hIS4%2Cete9j5Ene3Q%2CrYr8BqHTGww
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
aj7wJ8KD1aU%2CnZRI-WnDfmE%2CPUMm_PYi7b4%2ClZ4Z9xqgDSM%2CtWzLz6

In [126]:
not_trending_gb_df = process_youtube_requests(GB_trending_videoIds)

ViKX-5MoeaI%2COp3-bkAHReU%2CP-Wms1cB6uI%2CnD6eU4p1DKU%2CnB3nfc1tQ0E%2C6_U6_QGhIEM%2CyAhruSwum1c%2CIKuoOTRO1eo%2CZNcBZM5SvbY%2CtHmtxSB3mGo%2CP90o1kh3tVE%2CmvfzWFl5q9E%2C1GsY0y01b2Y%2C4NViHyjrNHw%2CzeU3lByZVWs%2CujA70PK6E7U%2CS-4rhjO6xYg%2CipQKBwQxoeQ%2CCYh_0QYQ-CE%2C8xAgsOaW1JM%2Ck9tgLnI0fFc%2CT9m_Ffxxxlc%2C1XoWqoKOjz4%2CX7wnqXo-tHA%2CnHL3lq7wskU%2CfEW-3E3fVJ0%2CExNKN05cU18%2CEZQhnrd57KQ%2CD2UE_Gw7XpI%2CzEVxP7z9scs%2COT0f-eSaf4M%2C5_3fBHN_MiI%2CXuJhzY87NhA%2Cc73Cu3TQnlg%2CFqBFX7pqDTc%2CngzC_8zqInk%2CxxiCUVo9MW0%2CEtwWqj9BKJA%2CSv_hGITmNuo%2CNkmr2-45UOM%2CsWcuhSlcgoY%2CWW_Z9UU8ynA%2C60L6D9rhmaw%2CSLsTskih7_I%2C5alLqZUfXLE%2ClTV1b1Qz3Yo%2C8Nk08sTpp3o%2CKVxFvjAhToA%2CJ2AvqK1ZpMY%2C7ncw_ycBJvE
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
mOFvJVroAJE%2Csa8l32GkhI4%2CvP1QKWi3xjk%2Cb46yOon49VI%2CtejoKyCgBsY%2C5cvYKWHa2oM%2CCtQJBnxYSxk%2C8-JV2NMjSAQ%2CU_3WyMiRDFg%2C08vIy0y1N9I%2CtyInv6RWL0Q%2Ci8tUwkcMVIE%2C7wMV_3YUetg%2CXvM8bgweH74%2CB1jiZ8-JLo0%2CWTDNEhDyKdo%2CEcqKSA

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
zxKRoCvX2yc%2Cdf9RU8mtYaM%2CGtTg3qYvLEE%2CEW0U2-bSjIw%2CDMT8gp_ESsU%2C-XuHWdSGOIY%2C8J4zJmvy_dM%2CYtJoHLwK9yg%2CxYIhxXt58uE%2CJZ0UEKL-8Sg%2CavhB6M-PT3Y%2CMPhAEcLaiNs%2CFlJLY0c0rts%2CTCq0K3DlFdc%2CYNsnHdIxovw%2CLmmV2b23DV0%2C0dak793Prs8%2C-ZjuAeRLjHM%2CXf3-aMPMdHg%2CgMcc8dKHSZs%2CBQ31l1gPGKM%2C-N1sjdLQIj8%2Cb2AcxL88DoI%2CO4etinsAy34%2CPyOniBaqtu4%2C9Sc-ir2UwGU%2Cg8pC2fUqvKY%2CJyqD_zfXfi8%2CBDxabu7EFcM%2Cs8n_cOeWBTs%2Cd-eSHPERoPk%2CF8dddZgcqIg%2CLsrrc_ru4CM%2CN18EwO-Rkxc%2C3xrQO-kAPxo%2Cdw0sjXiU4IE%2ChNdhoVW6Rhw%2C4CcfLHvjq8U%2CkAG13WKJN_E%2Cau2n7VVGv_c%2Couzj5yopegg%2CZSKEihd7Vlg%2COZ0SVkKboZg%2C0yoW_7DVq1g%2Ca8BOW-pqotg%2CExaoAv95D4I%2CUa5TvBFm3IM%2CinIOTWWvLVw%2CtizhQcPHYXs%2C701xgdCzExo
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
H52TTCoS7To%2C4NLfJqCnpHA%2CPHWAZNkqwN4%2C5JBx851kgLM%2CFxptfoHHyLM%2CkcRWyLMVrFM%2CVQH8ZTgna3Q%2C18eWTsM5X6Y%2C_pxRLV4i3CU%2Cbhj2s9FB

Something went wrong! 3
-i-XDXZl_jA%2CUpOntxHynSk%2CtQjsAJhsSw8%2ClkN51aqPOzU%2CylK5aBnat2Y%2C2J1VZwTsSQk%2ColyxlGVxktY%2C-P5Q38lf6cs%2CTr7LbAtVFBg%2C-AzoBkRCZ2o%2CNgueNhUiuJE%2C6wtwpUwxQik%2CLPw7SI2dxmk%2CawBbQhTXXWQ%2CLXiBei_iUes%2CqcDBnyN54rQ%2CKIX5gtGGZTE%2CZ7BShVPNpAU%2CPnwzUdOA9zQ%2CQtFWgkDEr94%2Ce0_96_YQu0k%2CdrV0QatqbRU%2C84hXRY0uwQ0%2CEyppxUfMdDg%2C181f8H8nUsw%2CYnsmoUntOzI%2CckIzS4u-fKs%2CC7LN_szqUmc%2Cnn6zXn-adhw%2CDqjZx4WOvZE%2Ce6HeD7jN3Mc%2Cu8h4Lsu_o6s%2CpJ14P-Xezeg%2CzfyEhAc7K_I%2CIglDs4i4No4%2CaVOOGDI-0kY%2CDjlaedAUzgQ%2CLPshYAeDyCM%2CHmslSJTCWDQ%2C4e-si815TfE%2CbWRt0MOrc3E%2C8kQD7t0eAcQ%2CpCjY3ZQY8WY%2CDSkrK0-EIVY%2CiXJyI2tJYgk%2C7DJNoVSUvAU%2CnyWAm6BpV5M%2CRM7lw0Ovzq0%2CZD3YZsD8cY8%2C1UqCuCFQUPc
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
8pg8FaFNeSI%2CUSXxCw3E980%2CTiixq9rT_J0%2COvRtf1g8C3g%2CajTuRQtm-eo%2CaLlNQS3P1UM%2CEjW7WzozgI0%2CWfB_JyAr5tE%2CwCrtk-pyP0I%2CHS9fcQfAeDc%2CrJ4LCmdmVfc%2Ck2R5FNOEEGw%2CRH0Bl2z7nE8%2C

Something went wrong! 3
Something went wrong! 3
u9L0pord5jE%2CYu_xrFKQcCA%2Cpy6PgXq0yDM%2CGHfBTlbhpiI%2C_RDVOYOW-rs%2CDpSWgZVIfi8%2Ckufxf2JiYyQ%2CfuPCIweVFMM%2C_htPRpsxiss%2CNe1uhw2sr28%2Cu2GdXLMM8Y8%2CGb7xq3th3wM%2CHvtb0DjZl94%2CUoVg2kNQL1U%2ChEcjgJSqSRU%2CwOS5VGkRPJk%2C0vbAafXy0cI%2C23y_1BDfqDk%2CgbPyFvAl4w4%2CDQYHCLz_bgw%2CuP8FnfotQhM%2CE_PIi8tRcCo%2CBQevgqg_8WI%2CTXx7ISST9PE%2CgbdRoizHAFU%2C12lj2kz3E9g%2CMgbngiiwihM%2Cmk1i0G7Nv-M%2C8s6syevtPhk%2CnceqQyqIa5o%2CVX5v4GUUM5M%2ChEdvvTF5js4%2Czy-RE76XDgM%2CILJcmFKgCfg%2C1v1LyQuY0kM%2CKIs_pgr5XLg%2CeOv4cLuw57k%2CtOqc5oqztBE%2CEqQHJdVq2R8%2CVQ-OgO00NSs%2CD9Z3rIjb1gI%2CQ7iOoOYvzu4%2C391c5nWG5-0%2CTAyel1PAcRE%2CQ0RVBTb6wTQ%2CujIBGLBlSbk%2C5x-d3pabd5o%2CrRgTMs_bGuI%2CLnoSCAi8Huc%2CDtB8YhsUkkg
wNxjZU_tOoc%2CRvA3q0ZU-NQ%2CXzaZXykn2SA%2CfN5HV79_8B8%2CTFto0jpD34c%2CiTPTrjQ_xPw%2CHhW3bmpEc8s%2C0XElmYomloA%2CFnX82yXRDmI%2CCxnJf0tWu48%2CmzB1VGEGcSU%2CQBRCegxuOko%2CbWt2URQxwFQ%2Cc9VQye6P8k0%2CizIyhdEHSPo%2C6IZK1k1elBU%2Cp8npDG2ulKQ%2CJEFO5RfzjTU%2Cjc

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
_WG1eI5xAz8%2CQnb2ZdoxbbU%2CFRPp1oQKxsQ%2CLbvE0FV_70U%2CKr3PoHOs-Qw%2CyxR-6yymA-M%2C8cJZ61x5N3g%2CT7VewKI44rQ%2CPXWRyvoYnic%2Ct8B4Yyo6NTM%2C7Rzk87oOfaE%2CBaiFRsPZe-8%2CuowrZJV6jVs%2CZ4H2W-k2j84%2CWHTPLpY8mBI%2ChanqTo3JZTs%2CC81FqB1zbQU%2CEcGFp3mgRHY%2CgyTeYHm-YDk%2C8ty7tfFU_kE%2C8PPDyEDYEss%2CAopjHz1zv3c%2CvpxbYH22DNY%2Cr8NMF2g5_Nw%2CNmtxt6GDzK8%2C6qRqcjB-iRE%2CSZDmuHSqwtg%2CMGJrVXcMaeg%2CD0YBSe1jHvo%2CxGBY3fNrYGk%2CFiexHGZILfU%2CqL4m04OyCkY%2CobvsSG6iUVU%2Ceo1JtiLCuHY%2CfQNZhs0QKq0%2CjR4lLJu_-wE%2CGH0wPmLIWow%2CymdJYiNRxcQ%2CFwblwcHBa6I%2C5x-d3pabd5o%2CWxMskhDXLYg%2C4h4fbuaU8Bw%2C6t6b7twnMM8%2CUVtWlePDY0o%2CbhOyXk11XDI%2CQqLgmequ7Bk%2ChyiX2YTWRO0%2CJMpuxLIgjPs%2CtccSuFBteAw%2CeqNAuHkKZGE
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
IElQwOB32XE%2CbSbWwn_YCr8%2CK0YEVNacmtw%2CV1-G89K1KWY%2CXcUiOy

In [38]:
not_trending_in_df = process_youtube_requests(IN_trending_videoIds)

y90ooB1Ow4A%2CFJgOqffQNpk%2CAoNjdleOn4M%2CvDF6eLSAVug%2C3GKstwNuzSc%2CpVkDZueTBpY%2Cjl2b9w9plBQ%2CM_A6T-Yuzzk%2CbCfZJI15zmw%2CmfTmL0B5cvU%2Cip3XozaiLeo%2CJyT51hDHThI%2CLFh9VW0wvr0%2CDGHRyFa3MAA%2CYb9yrKO7G9c%2CDxLNLaD61_4%2CZdotGhEpbJQ%2CU5iEpuQ-rT4%2Cimp9ZzKKiDQ%2CNuQupCVgzFg%2CUgyDqaUz6oA%2CCIJSIRW0Ga4%2CLnNv87fDrSc%2CDlDLsUp1nC8%2CA89zEyYrmjA%2CBvHwwpD_Oo8%2CTMrmOkMuv5I%2CF0dmbvepsXg%2CPjGMz852OSo%2CUextfOX0VgM%2CyouZZDszDLI%2CiXbRVNvl2LQ%2CIR8qCuR0X2s%2C2dHxnzkHTz8%2Cy_kN3hpc_tw%2C5XyVMYn7O20%2CWOotZeFQmUw%2CglOWHWuMEG0%2CDagl-IDZcpU%2CM0HrqwPavZQ%2CZW0CwumZe3E%2CRpOo61HZSWA%2C9ak7JGaQhSw%2CHhKfBMZl5u4%2CqQpGmDhxgRE%2CiQckakGluQ0%2Ci_B8iZHCQC0%2Cib06-pogOEs%2CmmOEBgamzlU%2CiiBlJnlDVQg
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
3KCnVa_Os7A%2CWhg26qAzKDM%2CLQxA15BNNKE%2CsMFV1MBj8XE%2CakwbjmLb3s4%2C2Hz9wwaJbvQ%2CLSGF9By_JUk%2C6eWLFt7EeuI%2Ck5chI_5-HmU%2C5wX9vrxUJ78%2CTATr6tDWPvE%2Cx2Fl3iz5tPw%2CqGzg2qOxnmI%2CzUo7_I3mSD0%2CMjnQHnlIZ8

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
dkXVfNmhnOE%2Cer_dkBZaiS8%2CsKgkBEU-FKo%2Cz2NvgU0dVx0%2Ck7MXSfoORTA%2CC9YF9dw5E3Q%2C4XMIsZUAyiM%2CJ3-FsuMS1GQ%2CrI0FCbEIs4A%2CkGgbap0iTBU%2C2h1yiK-uUgY%2C3J62ptNZDZE%2C4QAkdymQb5c%2CEkL-7WX87uk%2CWNcEX0N1-Hc%2C-Hi8T4bXzUQ%2CWgi5gwTqvbk%2CG0MoOOP8Pj8%2CPC8TXo2QmhA%2ChPvZdwv_RSo%2CgbtkqJVCpgM%2CAPwlLwPcyfI%2C57-LWmeSEZM%2CjaGZNgosNpk%2Cs7o0MED0gnw%2Ck5J9ZmnwUA0%2CXOq3_fh9lC4%2CQQ7_gVsBA40%2C-mn1bXdcJP0%2C2hEtg-EIQJE%2Czd-U6MaanYc%2C69GobtUoy2A%2CXrW0r3SkaPA%2CY3-PeuQ7nvY%2Ckt0V9yNu_Jo%2Cw8YVqBs6kyE%2Cf8Le5tl4KsM%2CuPgtO7BJGqw%2CWRYnzk04f-U%2CeHoSVRXgUCc%2CWwLPvneXD7k%2CSAVlJBfvc9M%2CBk-4AQIaQpw%2CXF2ocf5prqs%2C17l7RUe9Syk%2C3dKiw0NRrfo%2CoNfIlIOuh84%2CzaUHEzGgMbY%2CXmkLHUGXL2s%2CC9Og6ChNXfk
Something went wrong! 3
Something went wrong! 3
93p5h_hXdtA%2C7ZSRN8jRrOA%2CkduLmg9CXl8%2CHq_o7SurOag%2CrLazuvJ6oHM%2ChaYm5k6h5yc%2COsjW-9k-BiM%2CYHUddWp5Z48%2CxFX-1XdkX4k%2C1IFvi_N8kXs%2Cx012JMRtKag%2CHfe2

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
yVY7yBl6UEA%2C_bNrVP6CCfs%2Clm5NxGwY-Bg%2CPPB6dO7JOZU%2CNhLv_SXZErM%2C9GInoN_hhFk%2CBrLU0nXSfCs%2CMdFq3-TQZy8%2CElDPLY58pIM%2C8JYE0JGKQhI%2Ci3BiUodjHQU%2CUcRfZGTgHOc%2CPgVtuDLfxmI%2CAb-ODu5pLvs%2Ces8G3h1FEt4%2CpAsuOw4i5t0%2CucqEwhrHbeo%2CeTEsKT6b4Lk%2CF3tphqDwhu0%2COQQ1sGphUj4%2Ct3F9AouhNww%2CiS8qygJb1XU%2C7LVlQAYW3KE%2Cilji8UmTtOY%2CYY8_bE1laFc%2CZ0bzJCvmgLg%2CWjpUvtwe6mc%2Cd5boSdsLuqE%2C-P60cXWkaSU%2CQrI6bGo51f0%2Ces-5OLRrUf8%2C4nlqqofpMmI%2Cz7TL2qjWWSI%2CaYQYTOt-i9M%2Csv-crstX6Y8%2CdWzGSzapwJA%2Chp5CE9kArD8%2CdZ467GMqrKs%2CgjJjy28wXHM%2CoAZBvU4XtIA%2CQ-f84Ysu3-Q%2CqGMsDi_zAC0%2C7IwviQfTQGY%2CbzMiBV19tio%2C58fbxxZsPs0%2CBwYmaDSXdQY%2CEPwPoQA1KyM%2CAESUaI88oGY%2C6Hlga4N1J_s%2CQSdeQabXI-w
Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
j7isGMkt-ZQ%2Cs-y2xmk4ISk%2CWbPpPw-2MS

Something went wrong! 3
Something went wrong! 3
Something went wrong! 3
bF8hPk7y6T0%2CQY0DxmpSF7s%2CrsII-IIPFSQ%2Cz-KUX-kpeqQ%2CW0fKl43QmIE%2CVVwWL5PxXVI%2CXhXW68JGSfs%2CTJyy_qz_qII%2Cm1_4JFbedZE%2CiLSjDCIB99c%2CJN5rbJq1SFs%2C4QXjk0ILsPI%2COo76nzAY6QE%2CX09wDVW7JRI%2CZDlYzwEU6cs%2CLJ_dAaqqGG4%2CZwZ4YnAJIak%2CFXiaIH49oAU%2C9cbu6QgGniI%2Cpx98mwhp6XQ%2CJ2KIq0-ixkk%2C-qbiAAuygg8%2CrLGuf553a6s%2CktmcnBH-_oc%2Cc-UxoF7BkrM%2C_gQ4e1eRmBs%2C4dfqF4PO0NE%2Ctq7rAuPUPMo%2C5pfInvcv8O4%2Ck5xRIKbW2t4%2Cbfu0uqJyi_E%2CtBFNxO4VqoY%2CiOzjaGtYJEU%2Ckrolsw9KVFs%2C9MgfTY6wJBA%2CETmZR1kliqk%2C4cogdBU5sLc%2CsJLiwa5f00Q%2CpVqfSR8zJwc%2Cf9CeoDxybvc%2CTU0FrgRW6Sw%2CGHyk6i5id4M%2CdRVwJuvE3NE%2C6I-tH-OFkxw%2CubVH69V-D1E%2CoWG_BLt7vfU%2CSUOFCOq16m4%2C3mW_-8jGOMs%2CKpiVh_DY1GY%2CIMaiMGhJYGY
Something went wrong! 3
Something went wrong! 3
g-9MveCwsTk%2CrBv8PJbAuwg%2CsqJCcZixGWs%2CmZJiJ-M1YQY%2C1hHvH8xQDKA%2CRFVROHwWO6A%2CsQezaX5hIf0%2CnF3v61P8ZoQ%2Cc0ZC8PaOW7Q%2CfHcnhiW-Qok%2Cu7fNpj5s85Q%2C5SkXmH5UhiQ%2CGtZvMKdItX8%2C

In [330]:
list_of_all_nontrending_dfs = [not_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gb_df, not_trending_in_df]
for df in list_of_all_nontrending_dfs:
    df['video_id'] = df['video_id'].astype(str) 
    df['title'] = df['title'].astype(str)
    df['channel_title'] = df['channel_title'].astype(str)
    df['category_id'] = df['category_id'].astype(int)
    df['category'] = df['category'].astype(str)
    df['tags'] = df['tags'].astype(str)
    df['views'] = df['views'].astype(int)
    df['likes'] = df['likes'].astype(int)
    df['dislikes'] = df['dislikes'].astype(int)
    df['comment_count'] = df['comment_count'].astype(int)
    df['description'] = df['description'].astype(str)
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)
allDfsList = list_of_all_trending_dfs + list_of_all_nontrending_dfs + [full_trending_df] + [full_nontrending_df]
allDfsDf = pd.concat(allDfsList)
allDfsList.append(allDfsDf)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [332]:
def insert_category_field(df):
    # add a category column using the category id from the json file
    id_to_category = {}
    with open("youtube-new/US_category_id.json", 'r') as f: # the other category json files have missing category ids. We decided to just use the US file since it contained all of them and since they are standard internationally. 
        data = json.load(f)
        for category in data['items']:
            id_to_category[category['id']] = category['snippet']['title']
        categories = []
        for id in list(df['category_id']):
            categories.append(id_to_category[str(id)])
        df.insert(4, 'category', categories)

In [334]:
## insert new category field into dataframes
for df in allDfsList:
    insert_category_field(df)

In [335]:
def find_tags_stats(df):
    tags = list(df['tags'])
    tagsList = []
    for t in tags: 
        tagsList.append(len(t.split('|')))
    countTags = sum(tagsList)
    meanTags = sum(tagsList) / len(tagsList)
    minTags = min(tagsList)
    maxTags = max(tagsList)
    tagsStats = [countTags, meanTags, minTags, maxTags]
    return tagsStats

In [364]:
## output is a map containing all the numeric data for each df
describes = []
for df in allDfsList:
    describe = df.describe().loc[['count', 'mean', 'min', 'max'], ['views', 'likes', 'dislikes', 'comment_count']]
    describe.insert(4, "tags", find_tags_stats(df), True) 
    describes.append(describe)
describesKeys = ['US_trending_df', 'CA_trending_df', 'DE_trending_df', 'FR_trending_df', 'GB_trending_df',
                'IN_trending_df', 'not_trending_us_df', 'not_trending_ca_df', 'not_trending_de_df', 'not_trending_fr_df',
                'not_trending_gb_df', 'not_trending_in_df', 
                'full_trending_df', 'full_nontrending_df', 'allDfsDf']
describeMap = dict(zip(describesKeys, describes))

In [366]:
describeMap
### lets visualize this since this is all our numeric data!! 

{'US_trending_df':               views         likes      dislikes  comment_count           tags
 count  4.094900e+04  4.094900e+04  4.094900e+04   4.094900e+04  808183.000000
 mean   2.360785e+06  7.426670e+04  3.711401e+03   8.446804e+03      19.736331
 min    5.490000e+02  0.000000e+00  0.000000e+00   0.000000e+00       1.000000
 max    2.252119e+08  5.613827e+06  1.674420e+06   1.361580e+06      69.000000,
 'CA_trending_df':               views         likes      dislikes  comment_count           tags
 count  4.088100e+04  4.088100e+04  4.088100e+04   4.088100e+04  800372.000000
 mean   1.147036e+06  3.958269e+04  2.009195e+03   5.042975e+03      19.578093
 min    7.330000e+02  0.000000e+00  0.000000e+00   0.000000e+00       1.000000
 max    1.378431e+08  5.053338e+06  1.602383e+06   1.114800e+06     124.000000,
 'DE_trending_df':               views         likes      dislikes  comment_count           tags
 count  4.084000e+04  4.084000e+04  4.084000e+04   4.084000e+04  733959.000

In [367]:
## do sentiment analysis on each of the tags
positive_words_df = pd.read_fwf('positivewords.txt')
negative_words_df = pd.read_fwf('negativewords.txt')

def extract_features(words):
    return dict([(word, True) for word in words.split()])

def build_sentiment_analysis_model():
    positive_words = positive_words_df['positivewords'].values.tolist()
    negative_words = negative_words_df['negativewords'].values.tolist()
    pos_feats = [(extract_features(f), 'positive') for f in positive_words ]
    neg_feats = [(extract_features(f), 'negative') for f in negative_words ]
    dataset = pos_feats + neg_feats
    random.shuffle(dataset)
    cutoff = int(0.80 * len(dataset))
    train_data = dataset[:cutoff]
    test_data = dataset[cutoff:]

    classifier = NaiveBayesClassifier.train(train_data)
    # print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    return classifier

def execute_model(tags):
    classifications = {}
    classifier = build_sentiment_analysis_model()
    classifier.show_most_informative_features(5)
    for tag in tags:
        classified = classifier.classify(extract_features(tag))
        classifications[tag] = classified
    return classifications

In [369]:
###get most common tags
def get_most_common_tags(country_df):
    tags = country_df['tags'].to_string(index=False, header=False)
    split_tags = [i.replace('"', '') for i in tags.split("|")]
    stop_words = stopwords.words('english')
    filtered_tags = [word for word in split_tags if word not in stop_words]
    fdist = FreqDist(split_tags)
    most_popular_tags = fdist.most_common(1000)
    return dict(most_popular_tags)

In [379]:
us_trending_most_common_tags = get_most_common_tags(US_trending_df)
ca_trending_most_common_tags = get_most_common_tags(CA_trending_df)
de_trending_most_common_tags = get_most_common_tags(DE_trending_df)
fr_trending_most_common_tags = get_most_common_tags(FR_trending_df)
gb_trending_most_common_tags = get_most_common_tags(GB_trending_df)
in_trending_most_common_tags = get_most_common_tags(IN_trending_df)

us_nontrending_most_common_tags = get_most_common_tags(not_trending_us_df)
ca_nontrending_most_common_tags = get_most_common_tags(not_trending_ca_df)
de_nontrending_most_common_tags = get_most_common_tags(not_trending_de_df)
fr_nontrending_most_common_tags = get_most_common_tags(not_trending_fr_df)
gb_nontrending_most_common_tags = get_most_common_tags(not_trending_gb_df)
in_nontrending_most_common_tags = get_most_common_tags(not_trending_in_df)

### do visualization on these most common tags!!!

In [ ]:
## do following for both trending and nontrending datasets and/or per country
## do good visualizations for all!!!

## find likes to dislikes ratio ---> shows where people have the greatest divide and where people agree the most 
## which categories are the most popular? do highest average amongst likes and views ----> shows which type of videos people enjoy the most
## most trending videos in each country? do average amongst likes, views ------> shows most trending videos per country
## how long did a video stay trending? how long did it take to become trending?
## which country has the most active participation and online presence across trending and nontrending? ----> min, max, std, quartiles, counts, etc
## correlation between views, likes, dislikes, and comments
## get most frequent tags ----> run sentiment analysis on tags, run LDA model on tags
## what makes a video trending? find the minimum number/qualities required to make a video trending given our data
    ## -> for ex: need 2000 views, 1000 likes, 1500 comments, needs to be about sports, and the tags have to give off a "happy" feeling